# Comparison of rna-seq output to be used by Treehouse

## Get data

### Get output generated by John Vivian

John ran these using python (what I think of a the toil approach, rather than the docker approach). He also says: I used isolated virtualenvs for each version, and serially ran the same sample in each testing environment. I ran everything twice and got the exact same results each time. 

A note about the following commands: I added sudo because my svn settings are messed up. Also, I have to run some (any) sudo command in terminal and then run these cells before my administative privileges expire (15 min, I think). 

In [4]:
%%bash
sudo /usr/bin/svn export https://github.com/jvivian/ipython_notebooks.git/trunk/toil-rnaseq-version-concordance/TEST_208_EGAR/RSEM/rsem_genes.results jv_TEST_208_SJINF053_rsem_genes.results


A    jv_TEST_208_SJINF053_rsem_genes.results
Export complete.


In [5]:
%%bash
sudo /usr/bin/svn export https://github.com/jvivian/ipython_notebooks.git/trunk/toil-rnaseq-version-concordance/TEST_302_EGAR/RSEM/rsem_genes.results  jv_TEST_302_SJINF053_rsem_genes.results


A    jv_TEST_302_SJINF053_rsem_genes.results
Export complete.


### Get output generated and collected by Rob Currie
The following scp commands were run in terminal so I could put my password in

In [2]:
%%bash
# scp bazaar.pod:/scratch/rcurrie/old/3.0.2/SJINF053/RSEM/rsem_genes.results rc_3.0.2.core_SJINF053_rsem_genes.results
# scp bazaar.pod:/scratch/rcurrie/old/2.0.8A/SJINF053/rnaseq/RSEM/rsem_genes.results rc_2.0.8A_treehouse_SJINF053_rsem_genes.results
# scp bazaar.pod:/scratch/rcurrie/old/2.0.8B/SJINF053/rnaseq/RSEM/rsem_genes.results rc_2.0.8B_treehouse_SJINF053_rsem_genes.results



## Minimal comparison
This approach considers only the estimated_count values. It counts and characterizes the differences in values between multiple pairs of files. 


In [23]:
%%bash
# comparisons
comparisons="jv_TEST_208_SJINF053_rsem_genes.results+jv_TEST_302_SJINF053_rsem_genes.results 
rc_2.0.8A_treehouse_SJINF053_rsem_genes.results+rc_2.0.8B_treehouse_SJINF053_rsem_genes.results
rc_2.0.8A_treehouse_SJINF053_rsem_genes.results+rc_3.0.2.core_SJINF053_rsem_genes.results
jv_TEST_208_SJINF053_rsem_genes.results+rc_2.0.8A_treehouse_SJINF053_rsem_genes.results
jv_TEST_302_SJINF053_rsem_genes.results+rc_3.0.2.core_SJINF053_rsem_genes.results"

for i in $comparisons; do x=${i/+*} ; y=${i#*+} #; echo $x; echo; echo $y

# evaluations
# for colNum in {4..6} ; do
colNum=5
cat $x | cut -f $colNum >xVals
cat $y | cut -f $colNum >yVals
diff -y --suppress-common xVals yVals >zVals
cat zVals | awk ' BEGIN { FS="|" } { print ($2 - $1) }' >zDifference
maxVal=`cat zDifference | Rscript -e 'options(scipen=999); max (abs(as.numeric (readLines ("stdin"))))' | cut -f2 -d" "`
pctl75Val=`cat zDifference | Rscript -e 'options(scipen=999); as.numeric(quantile(abs(as.numeric (readLines ("stdin"))), 0.75))' | cut -f2 -d" "`

# report
echo
echo Between ; echo $x; echo and ; echo ${y},; echo `wc -l zVals | cut -f1 -d " "`  `head -1 xVals` 'values differed by =<' $maxVal '(75% by =<' $pctl75Val').'
#done
done

echo 
echo 'Comparing results with no differences generates a warning that can be ignored ("no non-missing arguments to max; returning -Inf")'



Between
jv_TEST_208_SJINF053_rsem_genes.results
and
jv_TEST_302_SJINF053_rsem_genes.results,
234 expected_count values differed by =< 2 (75% by =< 0.98).

Between
rc_2.0.8A_treehouse_SJINF053_rsem_genes.results
and
rc_2.0.8B_treehouse_SJINF053_rsem_genes.results,
0 expected_count values differed by =< -Inf (75% by =< NA).

Between
rc_2.0.8A_treehouse_SJINF053_rsem_genes.results
and
rc_3.0.2.core_SJINF053_rsem_genes.results,
1013 expected_count values differed by =< 9.44 (75% by =< 1).

Between
jv_TEST_208_SJINF053_rsem_genes.results
and
rc_2.0.8A_treehouse_SJINF053_rsem_genes.results,
16129 expected_count values differed by =< 51965.4 (75% by =< 24.97).

Between
jv_TEST_302_SJINF053_rsem_genes.results
and
rc_3.0.2.core_SJINF053_rsem_genes.results,
16148 expected_count values differed by =< 51965.4 (75% by =< 24.9725).

Comparing results with no differences generates a warning that can be ignored ("no non-missing arguments to max; returning -Inf")


Warning message:
In max(abs(as.numeric(readLines("stdin")))) :
  no non-missing arguments to max; returning -Inf


## Compare files with detailed output

This approach generates more detailed output than the minimal version above. Consider limiting it to one column at a time for more tractable output. 

In [24]:
%%bash
x=rc_3.0.2.core_SJINF053_rsem_genes.results
y=jv_TEST_302_SJINF053_rsem_genes.results
# inputs
# for colNum in {3..6} ; do
for colNum in {5..6} ; do
# evaluations
cat $x | cut -f $colNum >xVals
cat $y | cut -f $colNum >yVals
diff -y --suppress-common xVals yVals >zVals
cat zVals | awk ' BEGIN { FS="|" } { print ($2 - $1) }' >zDifference
# report
echo; echo '###'; echo '### COMPARISON'; echo '###'
echo
echo  "comparing the column titled " `head -1 xVals` in $x 
echo "to the column titled" `head -1 yVals` in $y

echo 
echo line counts of input files
wc -l $x; wc -l $y
echo; echo first 10 pairs of differing values in files
head zVals
echo; echo last 10 pairs of differing values in files
tail zVals
echo; echo number of different values in files
wc -l zVals
echo; echo first 10 differences in values between files
head zDifference
echo; echo summary of difference in values between files
cat zDifference | Rscript -e 'options(scipen=999); summary (abs(as.numeric (readLines ("stdin"))))'
## try this:
maxVal=`cat zDifference | Rscript -e 'options(scipen=999); max (abs(as.numeric (readLines ("stdin"))))' | cut -f2 -d" "`
pctl75Val=`cat zDifference | Rscript -e 'options(scipen=999); as.numeric(quantile(abs(as.numeric (readLines ("stdin"))), 0.75))' | cut -f2 -d" "`
echo
echo `wc -l zVals | cut -f1 -d " "`  `head -1 xVals` 'values differed by =<' $maxVal '(75% by =<' $pctl75Val')'
done



###
### COMPARISON
###

comparing the column titled  expected_count in rc_3.0.2.core_SJINF053_rsem_genes.results
to the column titled expected_count in jv_TEST_302_SJINF053_rsem_genes.results

line counts of input files
60499 rc_3.0.2.core_SJINF053_rsem_genes.results
60499 jv_TEST_302_SJINF053_rsem_genes.results

first 10 pairs of differing values in files
300.00							      |	293.00
804.19							      |	790.41
513.81							      |	508.59
751.00							      |	743.00
223.00							      |	221.22
904.00							      |	896.00
3943.00							      |	3896.00
341.00							      |	335.00
3334.00							      |	3296.00
1470.00							      |	1448.00

last 10 pairs of differing values in files
155.59							      |	151.60
24.01							      |	23.30
3605.64							      |	3542.13
607.74							      |	597.18
65.00							      |	59.00
114.88							      |	106.14
19.83							      |	19.70
4.00							      |	3.00
1.66							      |	1.78
80.50							      |	76.96

number of different values in files
16113 z